In [1]:
# 파이토치

import torch
import torch.nn as nn

class SENet(nn.Module):
    def __init__(self, num_ch, r):
        super(SENet, self).__init__()
        self.GAP = nn.AdaptiveAvgPool2d((1,1))
        self.FCL1 = nn.Linear(in_features=num_ch, out_features=int(num_ch/r))
        self.FCL2 = nn.Linear(in_features=int(num_ch/r), out_features=num_ch)
        self.relu = nn.ReLU(True)
        self.sig = nn.Sigmoid()
        self.view = View(-1)

    def forward(self, x):
        x1 = self.GAP(x)
        x1 = self.FCL1(self.view(x1))
        x1 = self.relu(x1)
        x1 = self.FCL2(x1)
        x1 = self.sig(x1)
        return x * x1.unsqueeze(2).unsqueeze(3).expand_as(x)

class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()

        layers = []

        layers += [nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
                   nn.ReLU(inplace=True),
                   SENet(32, 4),
                   nn.MaxPool2d(kernel_size=2, stride=2),
                   nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
                   nn.ReLU(inplace=True),
                   SENet(64, 16),
                   nn.MaxPool2d(kernel_size=2, stride=2),
                   View(-1),
                   nn.Linear(in_features=3136, out_features=128),
                   nn.ReLU(inplace=True),
                   nn.Linear(in_features=128, out_features=10)]

        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

class View(nn.Module):

    def __init__(self, *shape):
        super(View, self).__init__()
        self.shape = shape

    def forward(self, x):
        return x.view(x.shape[0], *self.shape) # x.shape == (batch_size, channel, width, height)

if __name__ == '__main__':  # 이 파일이 메인으로 돌아갈때만 아래를 실행한다.

    from torchsummary import summary
    model = CNN()
    summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
              ReLU-2           [-1, 32, 28, 28]               0
 AdaptiveAvgPool2d-3             [-1, 32, 1, 1]               0
              View-4                   [-1, 32]               0
            Linear-5                    [-1, 8]             264
              ReLU-6                    [-1, 8]               0
            Linear-7                   [-1, 32]             288
           Sigmoid-8                   [-1, 32]               0
             SENet-9           [-1, 32, 28, 28]               0
        MaxPool2d-10           [-1, 32, 14, 14]               0
           Conv2d-11           [-1, 64, 14, 14]          18,496
             ReLU-12           [-1, 64, 14, 14]               0
AdaptiveAvgPool2d-13             [-1, 64, 1, 1]               0
             View-14                   

In [27]:
# 케라스

from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Activation, GlobalAveragePooling2D, Reshape, multiply
from keras.models import Model

def SEmodule(pre_layer, ch, r):
    x = GlobalAveragePooling2D()(pre_layer)
    x = Dense(int(ch/r), activation='relu')(x)
    x = Dense(ch, activation='sigmoid')(x)
    x = Reshape((1, 1, ch))(x)
    x = multiply([pre_layer,x])
    
    return x

inputs = Input(shape=(28,28,1))

x = Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='same')(inputs)
x = Activation('relu')(x)
x = SEmodule(x, 32, 4)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(x)
x = Activation('relu')(x)
x = SEmodule(x, 64, 16)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)

outputs = Dense(10, activation='relu')(x)

SENet = Model(inputs=inputs, outputs=outputs)

SENet.summary()

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_26 (InputLayer)           (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_28 (Conv2D)              (None, 28, 28, 32)   320         input_26[0][0]                   
__________________________________________________________________________________________________
activation_27 (Activation)      (None, 28, 28, 32)   0           conv2d_28[0][0]                  
__________________________________________________________________________________________________
global_average_pooling2d_26 (Gl (None, 32)           0           activation_27[0][0]              
___________________________________________________________________________________________